In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.13.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


In [2]:
function p(k::Int, n::Int) # power sum symmetric polynomial
    @req n >= 0 "n >= 0 required"
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return p(R, k, n)
end

function p(R::ZZMPolyRing, k::Int, n::Int)
    x = gens(R)[1:n]
    return sum(x[i]^k for i in 1:n)
end

function e(k::Int,n::Int) # elementary symmetric polynomial
    @req n >= 0 "n >= 0 required"
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return e(R, k, n)
end

function e(R::ZZMPolyRing, k::Int, n::Int)
    return schur_polynomial(R, partitions(k)[length(partitions(k))], n) #schur_polynomial in Oscar 
end

function h(k::Int, n::Int) # complete homogeneous symmetric polynomial
    @req n >= 0 "n >= 0 required"
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return h(R, k, n)
end

function h(R::ZZMPolyRing, k::Int, n::Int)
    return schur_polynomial(R, partition(k), n)
end

function schur(lambda::Vector{Int}, n::Int = length(lambda)) # Schur polynomial
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return schur(R, lambda, n)
end

function schur(R::ZZMPolyRing, lambda::Vector{Int}, n::Int = length(lambda))
    return schur_polynomial(R, partition(lambda), n)
end

schur (generic function with 4 methods)

In [3]:
function schur_expansion(f) # expand the symmetric polynomial as a linear combination of Schur polynomials
    R = parent(f)
    n = length(vars(f))  # Number of variables
    result = []  # List to store the results
    while f != zero(f)
        l = filter!(x -> x != 0, collect(exponents(f))[end])
        c = collect(coefficients(f))[end]
        push!(result, (c, reverse(l)))
        f -= c * schur(R, reverse(l), n)
    end
    return result
end

function pieri_rule(k::Int, lambda::Vector{Int}) # Pieri rule
    n = 1 + length(lambda)
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return schur_expansion(h(R, k, n) * schur(R, lambda, n))
end

function dual_pieri_rule(k::Int, lambda::Vector{Int}) # dual Pieri rule
    n = k + length(lambda)
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return schur_expansion(e(R, k, n) * schur(R, lambda, n))
end

function murnaghan_nakayama_rule(k::Int, lambda::Vector{Int}) # Murnaghan-Nakayama rule
    n = k + length(lambda)
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return schur_expansion(p(R, k, n) * schur(R, lambda, n))
end

function littlewood_richardson_rule(lambda::Vector{Int}, mu::Vector{Int}, n::Int=length(lambda)+length(mu))
    # Littlewood-Richardson rule
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return schur_expansion(schur(R, lambda, n) * schur(R, mu, n))
end

littlewood_richardson_rule (generic function with 2 methods)

In [4]:
function grothendieck(lambda::Vector{Int}, n::Int = length(lambda)) 
    # (symmetric, Grassmann) Grothendieck polynomial
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return grothendieck(R, lambda, n)
end

function grothendieck(R::ZZMPolyRing, lambda::Vector{Int}, n::Int = length(lambda))
    if n == 0 || n < length(lambda)
        if isempty(lambda)
            return one(R)
        else
            return zero(R)
        end
    end
    @req n <= nvars(R) "n <= nvars(R) required"
    return grothendieck_polynomial_bf(R, lambda, n) #bialternant formula
end

function grothendieck_polynomial_bf(R::ZZMPolyRing, lambda::Vector{Int}, n::Int = length(lambda))
    @req n >= length(lambda) "number of variables must be at least the length of the partition"
    while length(lambda) < n
        push!(lambda, 0)  # Append zeros to the partition if needed
    end
    x = gens(R)[1:n]
    A = zero_matrix(R, n, n)
    for i = 1:n
        for j = 1:n
            A[i,j] = x[i]^(lambda[j]+n-j)*(1-x[i])^(j-1)
        end
    end
    sp = det(A)
    # divide by the product
    for i = 1:n - 1
        for j = i + 1:n
            sp = divexact(sp, x[i] - x[j])
        end
    end
    return sp
end

grothendieck_polynomial_bf (generic function with 2 methods)

In [15]:
function grothendieck_expansion(f) # expand a symmetric polynomial as a linear combination of (symmetric) Grothenideck polynomials
    R = parent(f)
    n = length(vars(f))
    result = []
    while f != zero(f)
        l = filter!(x -> x != 0, collect(exponents(f))[end])
        c = collect(coefficients(f))[end]
        push!(result, (c, reverse(l)))
        f -= c * grothendieck(R, reverse(l), n)
    end
    return result
end

function G_pieri_rule(k::Int, lambda::Vector{Int}) # K-theoretic Pieri rule
    n = 1 + length(lambda)
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return grothendieck_expansion(h(R, k, n) * grothendieck(R, lambda, n))
end

function dual_G_pieri_rule(k::Int, lambda::Vector{Int}) # dual K-theoretic Pieri rule
    n = k + length(lambda)
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return grothendieck_expansion(e(R, k, n) * grothendieck(R, lambda, n))
end

function G_murnaghan_nakayama_rule(k::Int, lambda::Vector{Int}) # K-theoretic Murnaghan-Nakayama rule
    n = k + length(lambda)
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return grothendieck_expansion(p(R, k, n) * grothendieck(R, lambda, n))
end

function G_littlewood_richardson_rule(lambda::Vector{Int}, mu::Vector{Int}, n::Int=length(lambda)+length(mu))
    # Littlewood-Richardson rule
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return grothendieck_expansion(grothendieck(R, lambda, n) * grothendieck(R, mu, n))
end

G_littlewood_richardson_rule (generic function with 2 methods)

In [7]:
function dual_grothendieck(lambda::Vector{Int}, n::Int = length(lambda))
    # dual stable Grothendieck polynomial
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return dual_grothendieck(R, lambda, n)
end

function dual_grothendieck(R::ZZMPolyRing, lambda::Vector{Int}, n::Int = length(lambda))
    if n == 0 || n < length(lambda)
        if isempty(lambda)
            return one(R)
        else
            return zero(R)
        end
    end
    @req n <= nvars(R) "n <= nvars(R) required"
    return dual_grothendieck_polynomial_bf(R, lambda, n) #bialternant formula
end

function dual_grothendieck_polynomial_bf(R::ZZMPolyRing, lambda::Vector{Int}, n::Int = length(lambda))
    @req n >= length(lambda) "number of variables must be at least the length of the partition"
    while length(lambda) < n
        push!(lambda, 0)  # Append zeros to the partition if needed
    end
    x = gens(R)[1:n]
    A = zero_matrix(R,n,n)
    for j = 1:n
        A[1,j] = x[j]^(lambda[1]+n-1)
        for i = 2:n
            A[i,j] = sum(binomial(i+k-2,k)*x[j]^(lambda[i] + n - i - k) for k = 0:(lambda[i] + n - i))
        end
    end
    sp = det(A)
    # divide by the product
    for i = 1:n - 1
        for j = i + 1:n
            sp = divexact(sp, x[i] - x[j])
        end
    end
    return sp
end

dual_grothendieck_polynomial_bf (generic function with 2 methods)

In [6]:
function dual_grothendieck_expansion(f)
    R = parent(f)
    n = length(vars(f))
    result = []
    while f != zero(f)
        l = filter!(x -> x != 0, reverse(collect(exponents(f))[1]))
        c = collect(coefficients(f))[1]
        push!(result, (c, reverse(l)))
        f -= c * dual_grothendieck(R, reverse(l), n)
    end
    return result
end

function g_pieri_rule(k::Int, lambda::Vector{Int})
    n = 1 + length(lambda)
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return dual_grothendieck_expansion(h(R, k, n) * dual_grothendieck(R, lambda, n))
end

function dual_g_pieri_rule(k::Int, lambda::Vector{Int})
    n = k + length(lambda)
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return dual_grothendieck_expansion(e(R, k, n) * dual_grothendieck(R, lambda, n))
end

function g_murnaghan_nakayama_rule(k::Int, lambda::Vector{Int})
    n = k + length(lambda)
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return dual_grothendieck_expansion(p(R, k, n) * dual_grothendieck(R, lambda, n))
end

function g_littlewood_richardson_rule(lambda::Vector{Int}, mu::Vector{Int}, n::Int=length(lambda)+length(mu))
    # Littlewood-Richardson rule
    R, _ = polynomial_ring(ZZ, n, cached = false)
    return dual_grothendieck_expansion(dual_grothendieck(R, lambda, n) * dual_grothendieck(R, mu, n))
end

g_littlewood_richardson_rule (generic function with 2 methods)

In [25]:
dual_pieri_rule(3,[2,1])

4-element Vector{Any}:
 (1, [3, 2, 1])
 (1, [3, 1, 1, 1])
 (1, [2, 2, 1, 1])
 (1, [2, 1, 1, 1, 1])

In [27]:
dual_G_pieri_rule(3,[2,1])

8-element Vector{Any}:
 (1, [3, 2, 1])
 (1, [3, 1, 1, 1])
 (1, [2, 2, 1, 1])
 (1, [2, 1, 1, 1, 1])
 (1, [3, 2, 1, 1])
 (2, [3, 1, 1, 1, 1])
 (2, [2, 2, 1, 1, 1])
 (1, [3, 2, 1, 1, 1])

In [29]:
dual_g_pieri_rule(3,[2,1])

12-element Vector{Any}:
 (1, [3, 2, 1])
 (1, [3, 1, 1, 1])
 (1, [2, 2, 1, 1])
 (1, [2, 1, 1, 1, 1])
 (-2, [3, 2])
 (-3, [3, 1, 1])
 (-3, [2, 2, 1])
 (-4, [2, 1, 1, 1])
 (3, [3, 1])
 (3, [2, 2])
 (6, [2, 1, 1])
 (-4, [2, 1])

In [30]:
g_murnaghan_nakayama_rule(3,[2,1])

10-element Vector{Any}:
 (1, [5, 1])
 (-1, [3, 3])
 (-1, [2, 2, 2])
 (1, [2, 1, 1, 1, 1])
 (1, [3, 2])
 (1, [2, 2, 1])
 (-3, [2, 1, 1, 1])
 (1, [2, 2])
 (3, [2, 1, 1])
 (-2, [2, 1])

In [8]:
g_littlewood_richardson_rule([2,1],[2,1])

16-element Vector{Any}:
 (1, [4, 2])
 (1, [3, 3])
 (1, [4, 1, 1])
 (2, [3, 2, 1])
 (1, [2, 2, 2])
 (1, [3, 1, 1, 1])
 (1, [2, 2, 1, 1])
 (-1, [4, 1])
 (-3, [3, 2])
 (-3, [3, 1, 1])
 (-3, [2, 2, 1])
 (-1, [2, 1, 1, 1])
 (2, [3, 1])
 (1, [2, 2])
 (2, [2, 1, 1])
 (-1, [2, 1])

In [9]:
G_littlewood_richardson_rule([2,1],[2,1])

25-element Vector{Any}:
 (1, [4, 2])
 (1, [4, 1, 1])
 (1, [3, 3])
 (2, [3, 2, 1])
 (1, [3, 1, 1, 1])
 (1, [2, 2, 2])
 (1, [2, 2, 1, 1])
 (-1, [4, 3])
 (-3, [4, 2, 1])
 (-1, [4, 1, 1, 1])
 (-2, [3, 3, 1])
 (-2, [3, 2, 2])
 (-3, [3, 2, 1, 1])
 (-1, [2, 2, 2, 1])
 (2, [4, 3, 1])
 (1, [4, 2, 2])
 (2, [4, 2, 1, 1])
 (1, [3, 3, 2])
 (1, [3, 3, 1, 1])
 (2, [3, 2, 2, 1])
 (-1, [4, 3, 2])
 (-1, [4, 3, 1, 1])
 (-1, [4, 2, 2, 1])
 (-1, [3, 3, 2, 1])
 (1, [4, 3, 2, 1])

In [10]:
littlewood_richardson_rule([2,1],[2,1])

7-element Vector{Any}:
 (1, [4, 2])
 (1, [4, 1, 1])
 (1, [3, 3])
 (2, [3, 2, 1])
 (1, [3, 1, 1, 1])
 (1, [2, 2, 2])
 (1, [2, 2, 1, 1])

In [11]:
murnaghan_nakayama_rule(3,[2,1])

4-element Vector{Any}:
 (1, [5, 1])
 (-1, [3, 3])
 (-1, [2, 2, 2])
 (1, [2, 1, 1, 1, 1])

In [12]:
G_murnaghan_nakayama_rule(3,[2,1])

27-element Vector{Any}:
 (1, [5, 1])
 (-1, [3, 3])
 (-1, [2, 2, 2])
 (1, [2, 1, 1, 1, 1])
 (1, [4, 3])
 (-2, [2, 2, 2, 1])
 (1, [4, 4])
 (1, [3, 3, 2])
 (-2, [2, 2, 2, 2])
 (-3, [2, 2, 2, 1, 1])
 (1, [3, 3, 3])
 (1, [3, 3, 2, 1])
 (-3, [2, 2, 2, 2, 1])
 ⋮
 (1, [3, 3, 2, 1, 1])
 (-3, [2, 2, 2, 2, 2])
 (1, [3, 3, 3, 2])
 (1, [3, 3, 3, 1, 1])
 (1, [3, 3, 2, 2, 1])
 (1, [3, 3, 3, 3])
 (1, [3, 3, 3, 2, 1])
 (1, [3, 3, 2, 2, 2])
 (1, [3, 3, 3, 3, 1])
 (1, [3, 3, 3, 2, 2])
 (1, [3, 3, 3, 3, 2])
 (1, [3, 3, 3, 3, 3])